In [2]:
%pip install numpy pandas scikit-learn xgboost lightgbm requests tqdm joblib

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install nba_api

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install nba_api pandas scikit-learn numpy


Note: you may need to restart the kernel to use updated packages.


In [69]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
games = LeagueGameFinder(
    season_nullable="2024-25",
    league_id_nullable="00",
    season_type_nullable="Regular Season"
).get_data_frames()[0]

games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
games = games[games['MIN'] > 0]
games = games[~games['MATCHUP'].str.contains('All-Star', na=False)]


In [88]:
games[games['TEAM_ABBREVIATION']=='MIA']

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,a_home
5,22024,1610612748,MIA,Miami Heat,0022401190,2025-04-13,MIA vs. WAS,L,241,118,...,9,37,46,28,8,6,8,12,-1.0,1
39,22024,1610612748,MIA,Miami Heat,0022401177,2025-04-11,MIA @ NOP,W,240,153,...,16,44,60,36,12,5,9,13,49.0,0
79,22024,1610612748,MIA,Miami Heat,0022401160,2025-04-09,MIA @ CHI,L,240,111,...,11,28,39,28,11,3,13,10,-8.0,0
111,22024,1610612748,MIA,Miami Heat,0022401145,2025-04-07,MIA vs. PHI,W,241,117,...,10,37,47,26,7,6,13,17,12.0,1
141,22024,1610612748,MIA,Miami Heat,0022401131,2025-04-05,MIA vs. MIL,L,265,115,...,12,40,52,24,4,1,10,21,-6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2286,22024,1610612748,MIA,Miami Heat,0022400147,2024-11-02,MIA @ WAS,W,240,118,...,21,44,65,19,6,3,13,18,20.0,0
2329,22024,1610612748,MIA,Miami Heat,0022400122,2024-10-30,MIA vs. NYK,L,239,107,...,7,32,39,27,6,3,10,17,-9.0,1
2376,22024,1610612748,MIA,Miami Heat,0022400105,2024-10-28,MIA vs. DET,W,241,106,...,10,34,44,24,14,3,9,14,8.0,1
2389,22024,1610612748,MIA,Miami Heat,0022400088,2024-10-26,MIA @ CHA,W,242,114,...,13,30,43,33,8,6,13,19,8.0,0


In [74]:
from nba_api.stats.static import teams
from pprint import pprint

all_teams = teams.get_teams()
pprint(all_teams, width=200)

[{'abbreviation': 'ATL', 'city': 'Atlanta', 'full_name': 'Atlanta Hawks', 'id': 1610612737, 'nickname': 'Hawks', 'state': 'Georgia', 'year_founded': 1949},
 {'abbreviation': 'BOS', 'city': 'Boston', 'full_name': 'Boston Celtics', 'id': 1610612738, 'nickname': 'Celtics', 'state': 'Massachusetts', 'year_founded': 1946},
 {'abbreviation': 'CLE', 'city': 'Cleveland', 'full_name': 'Cleveland Cavaliers', 'id': 1610612739, 'nickname': 'Cavaliers', 'state': 'Ohio', 'year_founded': 1970},
 {'abbreviation': 'NOP', 'city': 'New Orleans', 'full_name': 'New Orleans Pelicans', 'id': 1610612740, 'nickname': 'Pelicans', 'state': 'Louisiana', 'year_founded': 2002},
 {'abbreviation': 'CHI', 'city': 'Chicago', 'full_name': 'Chicago Bulls', 'id': 1610612741, 'nickname': 'Bulls', 'state': 'Illinois', 'year_founded': 1966},
 {'abbreviation': 'DAL', 'city': 'Dallas', 'full_name': 'Dallas Mavericks', 'id': 1610612742, 'nickname': 'Mavericks', 'state': 'Texas', 'year_founded': 1980},
 {'abbreviation': 'DEN', '

In [91]:
games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
games = games[games['MIN'] > 0]
games = games[~games['MATCHUP'].str.contains('All-Star', na=False)]

playoff_teams = [
    # West
    1610612760,  # OKC Thunder
    1610612745,  # HOU Rockets
    1610612747,  # LAL Lakers
    1610612743,  # DEN Nuggets
    1610612746,  # LAC Clippers
    1610612750,  # MIN Timberwolves
    1610612744,  # GSW Warriors
    1610612763,  # MEM Grizzlies

    # East
    1610612739,  # CLE Cavaliers
    1610612738,  # BOS Celtics
    1610612752,  # NYK Knicks
    1610612754,  # IND Pacers
    1610612749,  # MIL Bucks
    1610612765,  # DET Pistons
    1610612753,  # ORL Magic
    1610612748  # MIA Heat
]

games = games[games['TEAM_ID'].isin(playoff_teams)]

games['a_home'] = games['MATCHUP'].str.contains(r' vs\.').astype(int)

a = games.rename(columns=lambda c: 'a_'+c if c not in ['GAME_ID','GAME_DATE'] else c)
b = games.rename(columns=lambda c: 'b_'+c if c not in ['GAME_ID','GAME_DATE'] else c)

merged = (
    a.merge(b, on=['GAME_ID','GAME_DATE'])
     .query('a_TEAM_ID != b_TEAM_ID')
)

merged = merged.sort_values(['a_TEAM_ID','GAME_DATE'])

# compute for Team A and Team B separately
merged['a_diff'] = merged['a_PTS'] - merged['b_PTS']
merged['a_diff_5gm'] = (
    merged
    .groupby('a_TEAM_ID')['a_diff']
    .rolling(window=5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

# similar for B side
merged['b_diff'] = merged['b_PTS'] - merged['a_PTS']
merged['b_diff_5gm'] = (
    merged
    .groupby('b_TEAM_ID')['b_diff']
    .rolling(window=5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

# new feature: difference of those pre-game averages
merged['diff_5gm'] = merged['a_diff_5gm'] - merged['b_diff_5gm']

# Create the binary target column based on whether Team A won (assuming 'W' represents a win)
merged['a_win'] = (merged['a_WL'] == 'W').astype(int)
X = merged[['diff_5gm','a_a_home']]
y = merged['a_win']
merged.head(100)[merged["a_TEAM_ABBREVIATION"]=="MIA"]





C:\Users\ducta\AppData\Local\Temp\ipykernel_7936\2757409085.py:68: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  merged.head(100)[merged["a_TEAM_ABBREVIATION"]=="MIA"]


,a_SEASON_ID,a_TEAM_ID,a_TEAM_ABBREVIATION,a_TEAM_NAME,GAME_ID,GAME_DATE,a_MATCHUP,a_WL,a_MIN,a_PTS,...,b_TOV,b_PF,b_PLUS_MINUS,b_a_home,a_diff,a_diff_5gm,b_diff,b_diff_5gm,diff_5gm,a_win


In [77]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, shuffle=False)
model = LogisticRegression().fit(X_train, y_train)
preds = model.predict_proba(X_val)[:,1]
print("AUC =", roc_auc_score(y_val, preds))

AUC = 0.6647342995169082


In [78]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
clf = LogisticRegression(max_iter=1000)
grid = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=5, n_jobs=-1)
grid.fit(X_train, y_train)

print("best C:", grid.best_params_['C'])
print("cv AUC:", grid.best_score_)
model = grid.best_estimator_

best C: 0.1
cv AUC: 0.8196476041445028


In [79]:
# 1. Lock in your best‐C model and check hold‑out AUC
model = grid.best_estimator_
val_preds = model.predict_proba(X_val)[:,1]
print("Hold‑out AUC =", roc_auc_score(y_val, val_preds))

# 2. (Optional but recommended) Calibrate if you see mis‑calibration
from sklearn.calibration import CalibratedClassifierCV
calib = CalibratedClassifierCV(model, method='isotonic', cv=5)
calib.fit(X_train, y_train)
val_preds_cal = calib.predict_proba(X_val)[:,1]
print("Calibrated AUC =", roc_auc_score(y_val, val_preds_cal))

# 3. Compute a best‑of‑7 series win‑probability function
from scipy.stats import binom

def series_win_prob(p):
    # P(win ≥4 out of 7) = ∑_{k=4}^7 C(7,k) p^k (1−p)^(7−k)
    return sum(binom.pmf(k, 7, p) for k in range(4, 8))

# 4. Example: get per‑game p for Team A vs B, then series p
#    (assuming you've built a feature row `x_feat` for A @ home first)
# Here we define a sample x_feat: use the most recent diff_5gm and set home indicator to 1.
x_feat = [merged['diff_5gm'].iloc[-1], 1]
p_game = model.predict_proba([x_feat])[0,1]   # or calib.predict_proba
p_series = series_win_prob(p_game)
print(f"P(game) = {p_game:.3f}, P(series) = {p_series:.3f}")


Hold‑out AUC = 0.663768115942029
Calibrated AUC = 0.6579710144927537
P(game) = 0.266, P(series) = 0.087


c:\Users\ducta\anaconda3\envs\cs418env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [80]:
# Given playoff_teams = [West seeds 1–8, then East seeds 1–8]
west = playoff_teams[:8]
east = playoff_teams[8:]

# Round 1: 1 vs 8, 2 vs 7, 3 vs 6, 4 vs 5 in each conference
first_round = [
    # East matchups
    (east[0], east[7]),  # 1 vs 8
    (east[1], east[6]),  # 2 vs 7
    (east[2], east[5]),  # 3 vs 6
    (east[3], east[4]),  # 4 vs 5

    # West matchups
    (west[0], west[7]),  # 1 vs 8
    (west[1], west[6]),  # 2 vs 7
    (west[2], west[5]),  # 3 vs 6
    (west[3], west[4]),  # 4 vs 5
]

In [81]:
print("Playoffs:", playoff_teams)

Playoffs: [1610612760, 1610612745, 1610612747, 1610612743, 1610612746, 1610612750, 1610612744, 1610612763, 1610612739, 1610612738, 1610612752, 1610612754, 1610612749, 1610612765, 1610612753, 1610612748]


In [82]:
seed_ids = set([t for pair in first_round for t in pair])

# which IDs you actually have diffs for:
have_ids = set(last_diff.keys())

print("Missing from last_diff:", seed_ids - have_ids)

Missing from last_diff: {1610612739, 1610612746, 1610612747, 1610612748, 1610612750, 1610612754, 1610612765}


In [83]:
raw_games = leaguegamefinder.LeagueGameFinder(
    season_nullable="2024-25",
    league_id_nullable="00"
).get_data_frames()[0]

# 1) Drop DNPs
df = raw_games[raw_games['MIN'] > 0].copy()

# 2) Flag home/away
df['home'] = df['MATCHUP'].str.contains(r' vs\.').astype(int)

# 3) Merge in opponent points
opp = df[['GAME_ID','PTS']].rename(columns={'PTS':'opp_PTS'})
df = df.merge(opp, on='GAME_ID')

# 4) Per‑game diff & 5‑game rolling mean
df['diff'] = df['PTS'] - df['opp_PTS']
df = df.sort_values(['TEAM_ID','GAME_DATE'])
df['diff_5gm'] = (
    df
    .groupby('TEAM_ID')['diff']
    .rolling(window=5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

# 5) Final rolling diff for every team
last_diff = df.groupby('TEAM_ID')['diff_5gm'].last().to_dict()

In [84]:
from scipy.stats import binom

def series_win_prob(p):
    return sum(binom.pmf(k, 7, p) for k in range(4, 8))

last_diff = merged.groupby('a_TEAM_ID')['diff_5gm'].last().to_dict()

series_p = {}
for A, B in first_round:
    d = last_diff[A] - last_diff[B]
    p_game = model.predict_proba([[d, 0]])[0, 1]
    series_p[(A, B)] = series_win_prob(p_game)

c:\Users\ducta\anaconda3\envs\cs418env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ducta\anaconda3\envs\cs418env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ducta\anaconda3\envs\cs418env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ducta\anaconda3\envs\cs418env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ducta\anaconda3\envs\cs418env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticReg

In [85]:
import random
from collections import Counter

# helper to simulate one best‑of‑7
def simulate_series(A, B):
    return A if random.random() < series_p[(A, B)] else B

# simulate an entire bracket
def simulate_bracket():
    # Round 1: eight series in first_round order
    r1 = [simulate_series(A, B) for A,B in first_round]

    # Round 2: East winners vs each other, then West
    east_sf = [simulate_series(r1[0], r1[1]), simulate_series(r1[2], r1[3])]
    west_sf = [simulate_series(r1[4], r1[5]), simulate_series(r1[6], r1[7])]

    # Conference Finals
    east_f = simulate_series(east_sf[0], east_sf[1])
    west_f = simulate_series(west_sf[0], west_sf[1])

    # NBA Finals
    return simulate_series(east_f, west_f)

# run N brackets
N = 20_000
champs = Counter(simulate_bracket() for _ in range(N))

# show each team’s championship probability
for team_id, wins in champs.items():
    print(team_id, wins/N)

KeyError: (1610612748, 1610612753)